In [ ]:
%%file dataset.py
# Generate training dataset and test dataset.
from constant import *
import os
import numpy as np
from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import copy
from matplotlib.pyplot import imshow
import random
#%matplotlib inline

#### Parameters. ####
type_str = 'nocrop_obj'
# type_str =  # crop / nocrop / crop_obj / nocrop_obj / aperture
#dataset = [(0.0, 0), (1.0/4, 4), (1.0/3, 3), (1.0/2, 3), (2.0/3, 3), (4.0/5, 3), (9.0/10, 3), (1.0, 1)]
#dataset = [(0.0, 0), (0.1, 10), (0.2, 5), (0.3, 4), (0.4, 3), (0.5, 3), (0.6, 3), (0.7, 3), (0.8, 3), (0.9, 3), (1.0, 1)]
dataset = [(0.0, 0), (0.2, 9), (0.4, 9), (0.6, 9), (0.8, 9), (1.0, 9)] # when crop_obj or nocrop_obj, (size, total_num)

# obj image: always 1024 * 768 pixels
obj_width = 1024
obj_height = 768
obj_ratio = float(obj_width) / obj_height
#####################

# Load labels.
imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
label_to_wnid = np.loadtxt(imagenet_labels_filename, str, delimiter='\t')
wnid_to_label = {}
for label in range(len(label_to_wnid)):
    wnid = label_to_wnid[label].split(' ')[0]
    wnid_to_label[wnid] = label

training_dataset = dataset
test_dataset = dataset

train_folders = []
test_folders = []
train_files = []
test_files = []

for (slider_size, slider_num) in training_dataset:
    percent = str(int(100 * slider_size))
    f = open('{}dataset/train_{}_{}.txt'.format(imagenet_root, type_str, percent), 'w')
    train_files.append(f)
    
    folder = '{}dataset/train_{}_{}/'.format(imagenet_root, type_str, percent)
    if not os.path.exists(folder):
        os.makedirs(folder)
    train_folders.append(folder)
        

for (slider_size, slider_num) in test_dataset:
    percent = str(int(100 * slider_size))
    f = open('{}dataset/test_{}_{}.txt'.format(imagenet_root, type_str, percent), 'w')
    test_files.append(f)

    folder = '{}dataset/test_{}_{}/'.format(imagenet_root, type_str, percent)
    if not os.path.exists(folder):
        os.makedirs(folder)
    test_folders.append(folder)
    
if type_str == 'crop_obj' or type_str == 'nocrop_obj':
    obj_images = []
    for image_name in os.listdir(shapenet_root + 'object_nobg/'):
        img = Image.open(shapenet_root + 'object_nobg/' + image_name)
        #img = img.convert("RGBA")
        obj_images.append(img)

# occluder size = slider_size * slider_size
# occluder num = slider_num * slider_num
# path = 'imagenet_root/dataset/train_0/name'
# {wnid_imgid}_{crop/nocrop}_{rect_i(ifcrop)}_{slider_size}_{i}_{j}
def generate_datum(img_orig, path, f, class_id, rects, slider_size, slider_num):
    if type_str == 'crop':
        if slider_size == 0:
            for rect_i, rect in enumerate(rects):
                img = copy.copy(img_orig)
                img = img.crop(rect)
                datum_path = '{}_{}_{}_0_0_0.jpeg'.format(path, type_str, rect_i)
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for rect_i, rect in enumerate(rects):
                for i in range(slider_num):
                    for j in range(slider_num):
                        img = copy.copy(img_orig)
                        d = ImageDraw.Draw(img)
                        if (slider_num == 1):
                            delta = 1
                        else:
                            delta = (1 - slider_size) / float(slider_num - 1)
                        subrect = [0, 0, 0, 0]
                        subrect[0] = rect[0] + i * (rect[2] - rect[0]) * delta
                        subrect[1] = rect[1] + j * (rect[3] - rect[1]) * delta
                        subrect[2] = subrect[0] + (rect[2] - rect[0]) * slider_size
                        subrect[3] = subrect[1] + (rect[3] - rect[1]) * slider_size
                        d.rectangle(subrect, fill="black", outline=None)
                        img = img.crop(rect)
                        datum_path = '{}_{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)), str(i), str(j))
                        img.save(datum_path)
                        f.write('{} {}\n'.format(datum_path, str(class_id)))
    if type_str == 'nocrop':
        if slider_size == 0:
            datum_path = '{}_{}_0_0_0_0.jpeg'.format(path, type_str)
            img_orig.save(datum_path)
            f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for i in range(slider_num):
                for j in range(slider_num):
                    img = copy.copy(img_orig)
                    d = ImageDraw.Draw(img)
                    if (slider_num == 1):
                        delta = 1
                    else:
                        delta = (1 - slider_size) / float(slider_num - 1)
                    for rect in rects:
                        subrect = [0, 0, 0, 0]
                        subrect[0] = rect[0] + i * (rect[2] - rect[0]) * delta
                        subrect[1] = rect[1] + j * (rect[3] - rect[1]) * delta
                        subrect[2] = subrect[0] + (rect[2] - rect[0]) * slider_size
                        subrect[3] = subrect[1] + (rect[3] - rect[1]) * slider_size
                        d.rectangle(subrect, fill="black", outline=None)
                    datum_path = '{}_{}_{}_{}_{}_{}.jpeg'.format(path, type_str, 0, str(int(100 * slider_size)), str(i), str(j))
                    img.save(datum_path)
                    f.write('{} {}\n'.format(datum_path, str(class_id)))                    
    
    if type_str == 'crop_obj':
        if slider_size == 0:
            for rect_i, rect in enumerate(rects):
                img = copy.copy(img_orig)
                img = img.crop(rect)
                datum_path = '{}_{}_{}_{}_0.jpeg'.format(path, type_str, rect_i, int(100 * slider_size))
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for rect_i, rect in enumerate(rects):
                for num in range(slider_num):
                    img = copy.copy(img_orig)
                    random_obj = copy.copy(obj_images[random.randint(0, len(obj_images) - 1)])
                    width = rect[2] - rect[0]
                    height = rect[3] - rect[1]
                    ratio = float(width) / height
                    if ratio >= obj_ratio:
                        resize_scale = float(height) / obj_height * slider_size
                    else:
                        resize_scale = float(width) / obj_width * slider_size
                    random_obj = random_obj.resize((int(obj_width * resize_scale), int(obj_height * resize_scale)), Image.ANTIALIAS)
                    top_left = (random.randint(rect[0], rect[2] - random_obj.size[0]), \
                                random.randint(rect[1], rect[3] - random_obj.size[1]))
                    img.paste(random_obj, top_left, random_obj)
                    img = img.crop(rect)
                    datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, int(100 * slider_size), num)
                    img.save(datum_path)
                    f.write('{} {}\n'.format(datum_path, str(class_id)))
    if type_str == 'nocrop_obj':
        if slider_size == 0:
            datum_path = '{}_{}_{}_{}_0.jpeg'.format(path, type_str, 0, int(100 * slider_size))
            img_orig.save(datum_path)
            f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for num in range(slider_num):
                img = copy.copy(img_orig)
                for rect_i, rect in enumerate(rects):
                    random_obj = copy.copy(obj_images[random.randint(0, len(obj_images) - 1)])
                    width = rect[2] - rect[0]
                    height = rect[3] - rect[1]
                    ratio = float(width) / height
                    if ratio >= obj_ratio:
                        resize_scale = float(height) / obj_height * slider_size
                    else:
                        resize_scale = float(width) / obj_width * slider_size
                    random_obj = random_obj.resize((int(obj_width * resize_scale), int(obj_height * resize_scale)), Image.ANTIALIAS)
                    top_left = (random.randint(rect[0], rect[2] - random_obj.size[0]), \
                                random.randint(rect[1], rect[3] - random_obj.size[1]))
                    img.paste(random_obj, top_left, random_obj)
                datum_path = '{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, int(100 * slider_size), num)
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))              
                    
                    
    if type_str == 'aperture':
        if slider_size == 0: # All black.
            for rect_i, rect in enumerate(rects):
                img = copy.copy(img_orig)
                d = ImageDraw.Draw(img)
                d.rectangle(rect, fill="black", outline=None)
                img = img.crop(rect)
                datum_path = '{}_{}_{}_{}_0_0.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)))
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        elif slider_size == 1: # All visible. 
            for rect_i, rect in enumerate(rects):
                img = copy.copy(img_orig)
                img = img.crop(rect)
                datum_path = '{}_{}_{}_{}_0_0.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)))
                img.save(datum_path)
                f.write('{} {}\n'.format(datum_path, str(class_id)))
        else:
            for rect_i, rect in enumerate(rects):
                for i in range(slider_num):
                    for j in range(slider_num):
                        img = copy.copy(img_orig)
                        d = ImageDraw.Draw(img)
                        delta = (1 - slider_size) / float(slider_num - 1)
                        subrect = [0, 0, 0, 0]
                        subrect[0] = rect[0] + i * (rect[2] - rect[0]) * delta
                        subrect[1] = rect[1] + j * (rect[3] - rect[1]) * delta
                        subrect[2] = subrect[0] + (rect[2] - rect[0]) * slider_size
                        subrect[3] = subrect[1] + (rect[3] - rect[1]) * slider_size
                        d.rectangle([0, 0, img.size[0], subrect[1]], fill="black", outline=None)
                        d.rectangle([0, 0, subrect[0], img.size[1]], fill="black", outline=None)
                        d.rectangle([subrect[2], 0, img.size[0], img.size[1]], fill="black", outline=None)
                        d.rectangle([0, subrect[3], img.size[0], img.size[1]], fill="black", outline=None)
                        img = img.crop(rect)
                        datum_path = '{}_{}_{}_{}_{}_{}.jpeg'.format(path, type_str, rect_i, str(int(100 * slider_size)), str(i), str(j))
                        img.save(datum_path)
                        f.write('{} {}\n'.format(datum_path, str(class_id)))
            
        
synset_names = os.listdir(imagenet_root + 'image/')
             
for synset_index, synset_name in enumerate(synset_names):
    print "Processing synset [{}/{}]: {}".format(synset_index, len(synset_names), synset_name)
    image_names = os.listdir(imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images')
    annotation_names = os.listdir(imagenet_root + 'Annotation/' + synset_name + '/')
    n1 = [os.path.splitext(n)[0] for n in image_names]
    n2 = [os.path.splitext(n)[0] for n in annotation_names]
    intersection_names = list(set(n1) & set(n2))
    # train : test = 300 : 100
    for i in range(400):
        print 'Processing image [{}/{}]: {}'.format(i, 400, intersection_names[i])
        # Read bounding box.
        bbx_file = open(imagenet_root + 'Annotation/' + synset_name + '/' + intersection_names[i] + '.xml')
        xmltree = ET.parse(bbx_file)
        objects = xmltree.findall('object')
        rects = []
        for obj in objects:
            bbx = obj.find('bndbox')
            rects.append([int(it.text) for it in bbx])
            
        img_orig = Image.open(imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images/' + intersection_names[i] + '.JPEG')
        if i < 300: # Training dataset. 
            for index, (slider_size, slider_num) in enumerate(training_dataset):
                generate_datum(img_orig, '{}{}'.format(train_folders[index], intersection_names[i]), \
                               train_files[index], original_to_new_class_id[wnid_to_label[synset_name]], rects, slider_size, slider_num)
        else: # Testing dataset.
            for index, (slider_size, slider_num) in enumerate(test_dataset):
                generate_datum(img_orig, '{}{}'.format(test_folders[index], intersection_names[i]), \
                               test_files[index], original_to_new_class_id[wnid_to_label[synset_name]], rects, slider_size, slider_num)
            
for f in train_files:
    f.close()
for f in test_files:
    f.close()
            

In [ ]:
# Sample.

import os
import random
from constant import *

need_samples = ['10', '20', '30']

type_str = 'aperture'
sample_sum = 62316

# train
'''
for need_sample in need_samples:
    os.rename('{}dataset/train_{}_{}.txt'.format(imagenet_root, type_str, need_sample), '{}dataset/train_{}_{}_unsampled.txt'.format(imagenet_root, type_str, need_sample))
    f = open('{}dataset/train_{}_{}_unsampled.txt'.format(imagenet_root, type_str, need_sample), 'r')
    lines = f.readlines()
    f.close()
    sampled = random.sample(lines, sample_sum)
    f = open('{}dataset/train_{}_{}.txt'.format(imagenet_root, type_str, need_sample), 'w')
    f.writelines(sampled)
    f.close()
'''
# test
for need_sample in need_samples:
    os.rename('{}dataset/test_{}_{}.txt'.format(imagenet_root, type_str, need_sample), '{}dataset/test_{}_{}_unsampled.txt'.format(imagenet_root, type_str, need_sample))
    f = open('{}dataset/test_{}_{}_unsampled.txt'.format(imagenet_root, type_str, need_sample), 'r')
    lines = f.readlines()
    f.close()
    sampled = random.sample(lines, 21168)
    f = open('{}dataset/test_{}_{}.txt'.format(imagenet_root, type_str, need_sample), 'w')
    f.writelines(sampled)
    f.close()


In [3]:
# Create all.
import os
import random
from constant import *

dataset = [(0.0, 0), (0.2, 9), (0.4, 9), (0.6, 9), (0.8, 9), (1.0, 9)] # when crop_obj or nocrop_obj, (size, total_num)

sample_sum = {'train': {'crop_obj': 62316, 'nocrop_obj': 54000}, \
              'test': {'crop_obj': 21168, 'nocrop_obj': 18000}}

# train_{}_all
for type_str in ['crop_obj', 'nocrop_obj']:
    for func_str in ['train', 'test']:
        lines = []
        for size, num in dataset:
            percent = int(size * 100)
            f = open('{}dataset/{}_{}_{}.txt'.format(imagenet_root, func_str, type_str, percent), 'r')
            ls = f.readlines()
            f.close()
            lines = lines + ls
        f = open('{}dataset/{}_{}_all_unsampled.txt'.format(imagenet_root, func_str, type_str), 'w')
        f.writelines(lines)
        f.close()
        
        
        sampled = random.sample(lines, sample_sum[func_str][type_str])
        f = open('{}dataset/{}_{}_all.txt'.format(imagenet_root, func_str, type_str), 'w')
        f.writelines(sampled)
        f.close()

In [ ]:
# Make white color transparent.
from constant import *
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
%matplotlib inline
import numpy as np
import os

for image_name in os.listdir(shapenet_root + 'object_orig/'):
    print image_name
    img = Image.open(shapenet_root + 'object_orig/' + image_name)
    
    img = img.convert("RGBA")
    pixdata = img.load()
    for y in xrange(img.size[1]):
        for x in xrange(img.size[0]):
            if pixdata[x, y] == (255, 255, 255, 255):
                pixdata[x, y] = (255, 255, 255, 0)
    img.save(shapenet_root + 'object_nobg/' + image_name, 'PNG')


In [ ]:
# Make white color transparent.
from constant import *
from PIL import Image, ImageDraw
from matplotlib.pyplot import imshow
%matplotlib inline
import numpy as np
import os

image_names = os.listdir(shapenet_root + 'object_nobg/')
bg = Image.open(shapenet_root + 'object_nobg/' + image_names[0])
fg = Image.open(shapenet_root + 'object_nobg/' + image_names[1])

fg = fg.resize((800, 600))
bg.paste(fg, (0, 0), fg)

width = 2048
height = 2048


bg.thumbnail((width, height), Image.ANTIALIAS)

enlarge_ratio = max(float(width) / bg.size[0], float(height) / bg.size[1]) * 2
bg = bg.resize((int(bg.size[0] * enlarge_ratio), int(bg.size[1] * enlarge_ratio)), Image.ANTIALIAS)
bg.thumbnail((width, height), Image.ANTIALIAS)

imshow(np.asarray(bg))

In [ ]:
# Show typical images of each class.
from constant import *
import os
import numpy as np
#from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import copy
from matplotlib.pyplot import imshow
%matplotlib inline
from IPython.display import Image, display


imagenet_labels_filename = caffe_root + 'data/ilsvrc12/synset_words.txt'
with open(imagenet_labels_filename) as f:
    lines = f.readlines()
wnid_to_name = {}
for line in lines:
    wnid = line.split(' ')[0]
    name = line[(len(wnid) + 1):-1]
    wnid_to_name[wnid] = name

synset_names = os.listdir(imagenet_root + 'image/')
for synset_name in synset_names:
    image_names = os.listdir(imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images')
    print wnid_to_name[synset_name], 'http://image-net.org/synset?wnid=' + synset_name
    display(Image(filename=imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images/' + image_names[42]))
    display(Image(filename=imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images/' + image_names[32]))
    display(Image(filename=imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images/' + image_names[22]))
    display(Image(filename=imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images/' + image_names[12]))
    display(Image(filename=imagenet_root + 'image/' + synset_name + '/' + synset_name + '_original_images/' + image_names[0]))
    
    
    

In [2]:
# Sanity check.
import os
from constant import *

func_strs = ['train', 'test']
type_strs = ['nocrop_obj', 'crop_obj']
dataset = [(0.0, 0), (0.2, 9), (0.4, 9), (0.6, 9), (0.8, 9), (1.0, 9)] # when crop_obj or nocrop_obj, (size, total_num)

for func_str in func_strs:
    for type_str in type_strs:
        for size, num in dataset:
            percent = int(size * 100)
            path = imagenet_root + 'dataset/{}_{}_{}'.format(func_str, type_str, percent)
            files = os.listdir(path)
            print len(files), path

6000 /data2/haow3/data/imagenet/dataset/train_nocrop_obj_0
54000 /data2/haow3/data/imagenet/dataset/train_nocrop_obj_20
54000 /data2/haow3/data/imagenet/dataset/train_nocrop_obj_40
54000 /data2/haow3/data/imagenet/dataset/train_nocrop_obj_60
54000 /data2/haow3/data/imagenet/dataset/train_nocrop_obj_80
54000 /data2/haow3/data/imagenet/dataset/train_nocrop_obj_100
6924 /data2/haow3/data/imagenet/dataset/train_crop_obj_0
62316 /data2/haow3/data/imagenet/dataset/train_crop_obj_20
62316 /data2/haow3/data/imagenet/dataset/train_crop_obj_40
62316 /data2/haow3/data/imagenet/dataset/train_crop_obj_60
62316 /data2/haow3/data/imagenet/dataset/train_crop_obj_80
62316 /data2/haow3/data/imagenet/dataset/train_crop_obj_100
2000 /data2/haow3/data/imagenet/dataset/test_nocrop_obj_0
18000 /data2/haow3/data/imagenet/dataset/test_nocrop_obj_20
18000 /data2/haow3/data/imagenet/dataset/test_nocrop_obj_40
18000 /data2/haow3/data/imagenet/dataset/test_nocrop_obj_60
18000 /data2/haow3/data/imagenet/dataset/tes